## Множество с запросами суммы на отрезке

[условия.pdf](https://stepik.org/media/attachments/lesson/41233/statements.pdf)

[проверка результатов](https://stepik.org/lesson/45970/step/4)

Реализуйте структуру данных для хранения множества целых чисел, поддерживающую
запросы добавления, удаления, поиска, а также суммы на отрезке.

На вход в данной задаче будет дана последовательность таких запросов.
Чтобы гарантировать, что ваша программа обрабатывает каждый запрос по мере поступления
(то есть онлайн), каждый запрос будет зависеть от результата выполнения
одного из предыдущих запросов.
Если бы такой зависимости не было, задачу можно было бы решить оффлайн:
сначала прочитать весь вход и сохранить все запросы в каком-нибудь виде,
а потом прочитать вход ещё раз, параллельно отвечая на запросы.

#### Формат входа.

Изначально множество пусто. Первая строка содержит число запросов $n$.
Каждая из $n$ следующих строк содержит запрос в одном из следующих четырёх форматов:
- `+ i`: добавить число $f(i)$ в множество (если оно уже есть, проигнорировать запрос);
- `- i`: удалить число $f(i)$ из множества (если его нет, проигнорировать запрос);
- `? i`: проверить принадлежность числа $f(i)$ множеству;
- `s l r`: посчитать сумму всех элементов множества, попадающих в отрезок $[f(l),f(r)]$.

Функция $f$ определяется следующим образом.
Пусть $s$ — результат последнего запроса суммы на отрезке
(если таких запросов ещё не было, то $s = 0$). Тогда
    $$f(x) = (x + s) \mod 1 000 000 001$$

#### Формат выхода.

Для каждого запроса типа `? i` выведите `Found` или `Not found`.

Для каждого запроса суммы выведите сумму всех элементов множества,
попадающих в отрезок $[f(l),f(r)]$.
Гарантируется, что во всех тестах $f(l) \le f(r)$.

#### Ограничения.
$1 \le n \le 10^5$; $0 \le i \le 10^9$.

In [19]:
%matplotlib inline
from IPython.display import Image, HTML
from IPython.core.debugger import set_trace as trace

In [1]:
class SplitSumsBase:
    add = None
    remove = None
    find = None
    sumo = None
    last_sum = 0

    def do_task(self, task):
        def f(x):
            return (int(x) + self.last_sum) % 1000000001
        if task[0] == 's':
            op, left, right = task.split()
            left, right = f(left), f(right)
            s = self.sumof(left, right)
            self.last_sum = s
            return s
        op, val = task.split()
        val = f(val)
        if op == '?':
            ret = self.find(val)
            return ['Not found','Found'][ret]
        if op == '+':
            self.add(val)
        elif op == '-':
            self.remove(val)
    
    def run_tasks(self, tasks):
        results = []
        if isinstance(tasks, str):
            tasks = tasks.split(';')
        self.last_sum = 0
        for task in tasks:
            ret = self.do_task(task)
            if ret is not None:
                results.append(ret)
        return results

    def test(self, tasks, exam):
        if isinstance(exam, str):
            exam = exam.split(';')
        exam = [str(x) for x in exam]
        self.__init__()
        results = [str(x) for x in self.run_tasks(tasks)]
        assert results == exam, "\nneed {}\ngot  {}".format(results, exam)

In [2]:
def test_split_sums(SplitSums=SplitSumsBase):
    ss = SplitSums()
    ss.test('? 1;+ 1;? 1;+ 2;s 1 2;+ 1000000000;? 1000000000;- 1000000000;? 1000000000;s 999999999 1000000000;- 2;? 2;- 0;+ 9;s 0 9',
            'Not found;Found;3;Found;Not found;1;Not found;10')
    ss.test('? 0;+ 0;? 0;- 0;? 0',
            'Not found;Found;Not found')
    ss.test('+ 491572259;? 491572259;? 899375874;s 310971296 877523306;+ 352411209',
            'Found;Not found;491572259')
    print('ok')

In [3]:
class SplitSumsNaive(SplitSumsBase):
    def __init__(self):
        self.vals = set()

    def add(self, val):
        self.vals.add(val)
        return self
        
    def remove(self, val):
        if val in self.vals:
            self.vals.remove(val)
        return self

    def find(self, val):
        return val in self.vals
    
    def sumof(self, left, right):
        s = 0
        for val in self.vals:
            if left <= val <= right:
                s += val
        return s
    
test_split_sums(SplitSumsNaive)

ok


In [26]:
%%writefile tree-sums-naive.py

class SplitSumsNaive:
    def do_task(self, task):
        def f(x):
            return (int(x) + self.last_sum) % 1000000001
        if task[0] == 's':
            op, left, right = task.split()
            left, right = f(left), f(right)
            s = self.sumof(left, right)
            self.last_sum = s
            return s
        op, val = task.split()
        val = f(val)
        if op == '?':
            ret = self.find(val)
            return ['Not found','Found'][ret]
        if op == '+':
            self.add(val)
        elif op == '-':
            self.remove(val)
    
    def run_tasks(self, tasks):
        results = []
        if isinstance(tasks, str):
            tasks = tasks.split(';')
        self.last_sum = 0
        for task in tasks:
            ret = self.do_task(task)
            if ret is not None:
                results.append(ret)
        return results

    def test(self, tasks, exam):
        if isinstance(exam, str):
            exam = exam.split(';')
        exam = [str(x) for x in exam]
        self.__init__()
        results = [str(x) for x in self.run_tasks(tasks)]
        assert results == exam, "\nneed {}\ngot  {}".format(results, exam)

    def __init__(self):
        self.vals = set()
        self.last_sum = 0

    def add(self, val):
        self.vals.add(val)
        return self
        
    def remove(self, val):
        if val in self.vals:
            self.vals.remove(val)
        return self

    def find(self, val):
        return val in self.vals
    
    def sumof(self, left, right):
        s = 0
        for val in self.vals:
            if left <= val <= right:
                s += val
        return s        


def main():
    ssn = SplitSumsNaive()
    num_tasks = int(input())
    for _ in range(num_tasks):
        ret = ssn.do_task(input())
        if ret is not None:
            print(ret)

main()

Overwriting tree-sums-naive.py


In [28]:
%%bash
python3 tree-sums-naive.py <<EOF
15
? 1
+ 1
? 1
+ 2
s 1 2
+ 1000000000
? 1000000000
- 1000000000
? 1000000000
s 999999999 1000000000
- 2
? 2
- 0
+ 9
s 0 9
EOF
# expected:
#Not found
#Found
#3
#Found
#Not found
#1
#Not found
#10

Not found
Found
3
Found
Not found
1
Not found
10
